<a href="https://colab.research.google.com/github/princeedey/EXTRACTIVE-REVIEW-SUMMARIZER-USING-RNN/blob/main/Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
import os
import pickle
dicto = { 'Test1': 1, 'Test2': 2, 'Test3': 3 }
filename = "test_pkl_2.pkl"
with open(filename,'wb') as file:
      pickle.dump(dicto, file)
file.close()   



In [ ]:
import csv
from six.moves import cPickle as pickle
import numpy as np
x = []
with open('/content/gdrive/My Drive/Amazon.csv','r') as f:
     reader = csv.reader(f)  
     for line in reader: x.append(line)
with open('/content/test_pkl_2.pkl','w') as f:
     pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pandas as pd
from nltk.corpus import re
from nltk.corpus import stopwords
import pickle
import nltk
nltk.download('stopwords')
from keras.models import Sequential
from keras.layers import Dense
batch_size = 64
epochs = 110
latent_dim = 256
num_samples = 10000
stories =  pickle.load(open('/content/test_pkl_2.pkl', 'rb')) 
print('Loaded Stories %d' % len(stories))
print(type(stories))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Loaded Stories 568455
<type 'list'>


In [ ]:
#Vectorize
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for story in stories:
    input_text = story[9]
    for highlight in story[8]:
        target_text = highlight# We use "tab" as the "start sequence"
#character# for the targets, and "\n" as "end sequence"
        target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:input_characters.add(char)
        for char in target_text:
             if char not in target_characters:
                 target_characters.add(char)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
print(input_characters)
print(target_characters)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

['\x06', '\x07', '\x0c', '\x0f', '\x10', '\x13', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x82', '\x83', '\x87', '\x8c', '\x8e', '\x97', '\x99', '\x9e', '\xa2', '\xa3', '\xa5', '\xa6', '\xa7', '\xa8', '\xa9', '\xaa', '\xab', '\xac', '\xad', '\xae', '\xaf', '\xb0', '\xb1', '\xb2', '\xb4', '\xb5', '\xb7', '\xb8', '\xba', '\xbb', '\xbc', '\xbd', '\xbe', '\xc2', '\xc3']
['\t', '\n', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F

In [ ]:
def define_models(n_input, n_output, n_units):
# define training encoder
   encoder_inputs = Input(shape=(None, n_input))
   encoder = LSTM(n_units, return_state=True)
   encoder_outputs, state_h, state_c = encoder(encoder_inputs)
   encoder_states = [state_h, state_c]
# define training decoder
   decoder_inputs = Input(shape=(None, n_output))
   decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
   decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
   decoder_dense = Dense(n_output, activation='softmax')
   decoder_outputs = decoder_dense(decoder_outputs)
   model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# define inference encoder
   encoder_model = Model(encoder_inputs, encoder_states)
# define inference decoder
   decoder_state_input_h = Input(shape=(n_units,))
   decoder_state_input_c = Input(shape=(n_units,))
   decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
   decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
   decoder_states = [state_h, state_c]
   decoder_outputs = decoder_dense(decoder_outputs)
   decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
# return all models
   return model, encoder_model, decoder_model

In [ ]:
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    encodestate = infenc.predict(source)# start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)# collect predictions
    output = list()
    for t in range(n_steps):# predict next char
          yhat, h, c = infdec.predict([target_seq] + state)# store prediction
          output.append(yhat[0,0,:])# update state
          state = [h, c]# update target sequence
          target_seq = yhat
          return array(output)

In [ ]:
contractions = {"ain't": "am not","aren't": "are not","can't":"cannot","can't've": "cannot have","'cause": "because","could've": "couldhave","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have"}
def clean_text(text, remove_stopwords=True):# Convert words to lower case
    text = text.lower()
#if True:text = text.split()
    new_text = []
    for word in text:
        if word in contractions:
             new_text.append(contractions[word])
        else:
             new_text.append(word)
        text = " ".join(new_text)
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text)
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)
        if remove_stopwords:
             text = text.split()
             stops = set(stopwords.words("english"))
             text = [w for w in text if not w in stops]
             text = " ".join(text)
        return text


In [ ]:
clean_summaries = []
for summary in reviews.Summary:clean_summaries.append(clean_text(summary,
remove_stopwords=False))
print("Summaries are complete.")
clean_texts = []
# for text in reviews.Text:
# clean_texts.append(clean_text(text))
# print("Texts are complete.")
stories = list()
for i, text in enumerate(clean_summaries):
       stories.append({'story': text, 'highlights': clean_summaries[i]})# save
#to file
pickle.dump(stories, open('review_dataset.pkl', 'wb'))


Summaries are complete.


In [ ]:
import pandas as pd
from nltk.corpus import re
from nltk.corpus import stopwords
import pickle
import nltk
nltk.download('stopwords')
reviews = pd.read_csv("/content/gdrive/My Drive/Amazon.csv")
print(reviews.shape)
print(reviews.head())
print(reviews.isnull().sum())
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator', 'Score','Time'], 1)
reviews = reviews.reset_index(drop=True)
print(reviews.head())
for i in range(5):
    print("Review #",i+1)
    print(reviews.Summary[i])
    print(reviews.Text[i])
    print()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(568454, 10)
   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary  

In [ ]:
print(open('review_dataset.pkl', 'wb'))

<open file 'review_dataset.pkl', mode 'wb' at 0x7f3a670324b0>
